In [6]:
from parse_cpe_xml import *
from registry_query import *
#from similarity import *
from cpe_similarity import *

In [ ]:
software_inventory = get_installed_software()

example_cpe = "cpe:2.3:python:python:3.8.10:*:*:*:*:*:*:*:*"

cpe_vendor = example_cpe.split(":")[2]
cpe_name = example_cpe.split(":")[3]
cpe_version = example_cpe.split(":")[4]

best_match = [{},0]

for s in software_inventory:
    total_similarity = 0

    if s.get("Vendor") and len(s.get("Vendor")) > 0:
        vendor_similarity = string_similarity(cpe_vendor,s.get("Vendor"))
        total_similarity += vendor_similarity

    if s.get("Name") and len(s.get("Name")) > 0:
        name_similarity = string_similarity(cpe_name,s.get("Name"))
        total_similarity += name_similarity
    
    if s.get("Version") and len(s.get("Version")) > 0:
        version_similarity = string_similarity(cpe_version,s.get("Version"))
        total_similarity += version_similarity

    if total_similarity > best_match[1]:
        best_match = [s,total_similarity]

print(best_match)

    

In [2]:
software_inventory = get_installed_software()

In [3]:
cpe_dictionary = return_cpe_df()

In [ ]:
all_matches = []

for software in software_inventory[60:61]:
    best_match = [None,0]

    if software.get("Name"):
        name_importance_vector = importance(software["Name"])
    
    if software.get("Vendor"):
        vendor_importance_vector = importance(software["Vendor"])

    for i,cpe in enumerate(cpe_dictionary.name.to_list()):
        cpe_vendor = cpe.split(":")[2]
        cpe_name = cpe.split(":")[3]
        cpe_version = cpe.split(":")[4]

        total_similarity = 0

        if software.get("Vendor") and len(software.get("Vendor")) > 0:
            vendor_similarity_score = string_similarity_vector(cpe_vendor,software.get("Vendor"),vendor_importance_vector)
            total_similarity += vendor_similarity_score

        if software.get("Name") and len(software.get("Name")) > 0:
            name_similarity_score = string_similarity_vector(cpe_name,software.get("Name"),name_importance_vector)
            total_similarity += name_similarity_score

        if software.get("Version") and len(software.get("Version")) > 0:
            version_similarity_score = version_similarity(cpe_version,software.get("Version"))
            total_similarity += version_similarity_score

        if total_similarity > best_match[1] and "a" in cpe.split(":")[1]:
            best_match = [cpe,total_similarity]
    
    all_matches.append([software,best_match])

for match in all_matches:
    print(match)
        

In [9]:
all_matches = []

for software in software_inventory:
    best_match = [None,0]

    for cpe in cpe_dictionary.name.to_list():
        if software.get("Vendor") and software.get("Name") and software.get("Version"):
            similarity_score = return_similarity(cpe,software.get("Vendor"),software.get("Name"),software.get("Version"))
        
        else: continue

        if similarity_score > best_match[1] and "a" in cpe.split(":")[1]:
            best_match = [cpe,similarity_score]

    all_matches.append([software,best_match])

for match in all_matches:
    if match[1][0] != None: print(match[1][0])
